### Preuzimanje dataseta

Instaliranje potrebnih biblioteka

In [ ]:
pip install --upgrade pip

In [ ]:
!pip install tensorflow torch torchvision umap-learn

In [ ]:
!pip install tensorflow_datasets

In [ ]:
!pip install fiftyone

In [ ]:
!pip install "fiftyone[desktop]"

In [ ]:
pip install ultralytics

In [1]:
from ultralytics import YOLO
import tensorflow as tf
from pathlib import Path
from tqdm import tqdm
import shutil

Podskup slika iz OpenImagesv7 preuzet je korištenjem FiftyOne. 

In [4]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

Zbog smanjenja potrebnih resursa za preuzimanje i kasnije treniranje modela, preuzet je manji skup podataka, ukupno 100 slika. Skup za treniranje sadrži 70 slika, od kojih je 35 slika ovaca, a 35 slika sova. Skup za validaciju i skup za testiranje sadrže svaki po 15 slika.

Preuzeto je prvo 35 slika ovaca i 35 slika sova odvojeno, kako bi zastupljenost obje klase bila podjednaka. Potom su slike spojene u skup za treniranje. 

In [27]:
train_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections", "classifications"],
    classes=["Sheep"],
    max_samples=35,
    seed=51,
    shuffle=True,
    dataset_name="train-dataset",
)

Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading 'open-images-v7' split 'train'
 100% |███████████████████| 35/35 [993.0ms elapsed, 0s remaining, 35.2 samples/s]     
Dataset 'train-dataset' created


In [28]:
owl_subset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections", "classifications"],
    classes=["Owl"],
    max_samples=35,
    seed=51,
    shuffle=True,
    dataset_name="train-dataset-owls",
)

Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading 'open-images-v7' split 'train'
 100% |███████████████████| 35/35 [564.3ms elapsed, 0s remaining, 62.3 samples/s]   
Dataset 'train-dataset-owls' created


In [29]:
train_dataset.merge_samples(owl_subset)

In [30]:
val_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections", "classifications"],
    classes=["Sheep", "Owl"],
    max_samples=15,
    seed=51,
    shuffle=True,
    dataset_name="val-dataset",
)

Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v7' split 'validation'
 100% |███████████████████| 15/15 [210.8ms elapsed, 0s remaining, 72.2 samples/s]  
Dataset 'val-dataset' created


In [31]:
test_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="test",
    label_types=["detections", "classifications"],
    classes=["Sheep", "Owl"],
    max_samples=15,
    seed=51,
    shuffle=True,
    dataset_name="test-dataset",
)

Necessary images already downloaded
Existing download of split 'test' is sufficient
Loading 'open-images-v7' split 'test'
 100% |███████████████████| 15/15 [645.4ms elapsed, 0s remaining, 23.6 samples/s]      
Dataset 'test-dataset' created


### Transformacija u YOLO anotaciju

Budući da OpenImages i YOLO koriste različitu anotaciju objekata na slikama, potrebno je prilagoditi anotaciju u preuzetom datasetu u format prikladan za YOLO modele. Stvorena su tri nova direktorija za YOLO anotacije skupova za treniranje, validaciju i testiranje. Osim toga, stvorena je i config.yaml datoteka koja se pri treniranju modela šalje kao ulazni parametar i naznačava skupove za treniranje i validaciju.

Transformacija skupa za treniranje

In [61]:
output_dir = Path("yolo_train_dataset")
output_dir.mkdir(exist_ok=True) 

In [62]:
for sample in tqdm(train_dataset):
    img_path = sample.filepath
    img_filename = Path(img_path).name
    yolo_labels_path = output_dir / (Path(img_filename).stem + ".txt")

    with open(yolo_labels_path, "w") as f: 
        d = sample.detections.detections
        for el in d: 
            if el['label'] == 'Sheep': 
                bbox = el['bounding_box']
                x, y, width, height = bbox[0], bbox[1], bbox[2], bbox[3]
                x_center = x + width/2
                y_center = y + height/2
                yolo_label = f"0 {x_center} {y_center} {width} {height} \n"
                f.write(yolo_label)
            
            if el['label']=='Owl':
                bbox = el['bounding_box']
                x, y, width, height = bbox[0], bbox[1], bbox[2], bbox[3]
                x_center= x + width/2
                y_center = y + height/2
                yolo_label = f"1 {x_center} {y_center} {width} {height} \n"
                f.write(yolo_label)
            
    shutil.copy(img_path, output_dir / img_filename)

100%|██████████| 70/70 [00:00<00:00, 178.61it/s]


Transformacija skupa za validaciju

In [63]:
output_dir = Path("yolo_val_dataset")
output_dir.mkdir(exist_ok=True) 

In [64]:
for sample in tqdm(val_dataset):
    img_path = sample.filepath
    img_filename = Path(img_path).name
    yolo_labels_path = output_dir / (Path(img_filename).stem + ".txt")

    with open(yolo_labels_path, "w") as f: 
        d = sample.detections.detections
        for el in d: 
            if el['label'] == 'Sheep': 
                bbox = el['bounding_box']
                x, y, width, height = bbox[0], bbox[1], bbox[2], bbox[3]
                x_center = x + width/2
                y_center = y + height/2
                yolo_label = f"0 {x_center} {y_center} {width} {height} \n"
                f.write(yolo_label)
            
            if el['label']=='Owl':
                bbox = el['bounding_box']
                x, y, width, height = bbox[0], bbox[1], bbox[2], bbox[3]
                x_center= x + width/2
                y_center = y + height/2
                yolo_label = f"1 {x_center} {y_center} {width} {height} \n"
                f.write(yolo_label)
            
    shutil.copy(img_path, output_dir / img_filename)

100%|██████████| 15/15 [00:00<00:00, 125.43it/s]


Transformacija skupa za testiranje

In [65]:
output_dir = Path("yolo_test_dataset")
output_dir.mkdir(exist_ok=True) 

In [66]:
for sample in tqdm(test_dataset):
    img_path = sample.filepath
    img_filename = Path(img_path).name
    yolo_labels_path = output_dir / (Path(img_filename).stem + ".txt")

    with open(yolo_labels_path, "w") as f: 
        d = sample.detections.detections
        for el in d: 
            if el['label'] == 'Sheep': 
                bbox = el['bounding_box']
                x, y, width, height = bbox[0], bbox[1], bbox[2], bbox[3]
                x_center = x + width/2
                y_center = y + height/2
                yolo_label = f"0 {x_center} {y_center} {width} {height} \n"
                f.write(yolo_label)
            
            if el['label']=='Owl':
                bbox = el['bounding_box']
                x, y, width, height = bbox[0], bbox[1], bbox[2], bbox[3]
                x_center= x + width/2
                y_center = y + height/2
                yolo_label = f"1 {x_center} {y_center} {width} {height} \n"
                f.write(yolo_label)
            
    shutil.copy(img_path, output_dir / img_filename)

100%|██████████| 15/15 [00:00<00:00, 99.99it/s]


Za bilo koju sliku iz dataseta, može se pristupiti nizu detektiranih objekata u slici te za svaki objekt ispisati koordinate i labelu.

In [67]:
for sample in tqdm(test_dataset):
    d = sample.detections.detections 
    for el in d:
        print(str(el['label']) + " " +str(el['bounding_box']))

100%|██████████| 15/15 [00:00<00:00, 135.29it/s]

Sheep [0.29941002, 0.15265487, 0.6991150800000001, 0.84292038]
Goat [0.30678466, 0.15265487, 0.6932153400000001, 0.84734513]
Owl [0.42097488, 0.08388521, 0.36779912, 0.8741721899999999]
Owl [0.06776181, 0.028616853, 0.7720738899999999, 0.879173247]
Sheep [0.184375, 0.21041666, 0.6296875, 0.6041667]
Sheep [0.4875, 0.097765364, 0.5125, 0.810055876]
Pig [0.0, 0.4441341, 0.4975, 0.4804469]
Hat [0.44103393, 0.16565657, 0.18093696999999997, 0.29898989]
Human face [0.5024233, 0.22222222, 0.05977379999999999, 0.08080807999999998]
Human hair [0.46688208, 0.22020201, 0.11308562, 0.09696970999999999]
Sheep [0.42326334, 0.42828283, 0.29402259, 0.44646463000000003]
Human leg [0.11308562, 0.25454545, 0.16962845, 0.5090908999999999]
Human leg [0.31340873, 0.25454545, 0.11308562999999999, 0.5131313500000001]
Human leg [0.4394184, 0.5090909, 0.11147014, 0.2888889]
Human leg [0.58966076, 0.45858586, 0.07754439000000002, 0.27070704]
Fashion accessory [0.46849757, 0.7191919, 0.05492729000000002, 0.0929292